---

In [1]:
import numpy as np

### 感知器公式

![](https://raw.githubusercontent.com/massquantity/DL_from_scratch_NOTE/master/pic/1.png)

<font size=2> 
    b 是被称为偏置的参数,用于控制神经元被激活的容易程度; 而 w1 和 w2是表示各个信号的权重的参数,用于控制各个信号的重要性。</font>
<br><br><br>

### ReLU激活函数

In [1]:
def relu(x):
    return np.maximum(0, x)  # 不用 np.max

## 3层神经网络的实现

![](https://raw.githubusercontent.com/massquantity/DL_from_scratch_NOTE/master/pic/ch03_1.png)
![](https://raw.githubusercontent.com/massquantity/DL_from_scratch_NOTE/master/pic/ch03_2.png)
![](https://raw.githubusercontent.com/massquantity/DL_from_scratch_NOTE/master/pic/ch03_3.png)
![](https://raw.githubusercontent.com/massquantity/DL_from_scratch_NOTE/master/pic/ch03_4.png)
![](https://raw.githubusercontent.com/massquantity/DL_from_scratch_NOTE/master/pic/ch03_5.png)

In [ ]:
def init_network():
    network = dict()
    network['W1'] = np.random.randn(2, 3)
    network['b1'] = np.random.randn(3)
    network['W2'] = np.random.randn(2, 3)
    network['b2'] = np.random.randn(2)
    network['W3'] = np.random.randn(2, 2)
    network['b3'] = np.random.randn(2)
    return network

def forward(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    z1 = np.dot(x, W1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)
    z3 = np.dot(a2, W3) + b3
    y = softmax(z3)
    return y

### 实现 softmax 函数时的注意事项
<br>

<font size=2> 
    一般的 `softmax` 实现在计算机的运算上有一定的缺陷。这个缺陷就是溢出问题。`softmax` 函数的实现中要进行指
数函数的运算,但是此时指数函数的值很容易变得非常大。比如,$e^{10}$ 的值
会超过 20000,  $e^{100}$ 会变成一个后面有 40 多个 0 的超大值,$e^{1000}$ 的结果会返回
一个表示无穷大的 `inf`。如果在这些超大值之间进行除法运算,结果会出现“不确定”的情况，所以通常减去最大值 
</font>

In [3]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    y = exp_a / np.sum(exp_a)
    return y

### show image

In [11]:
import sys, os
sys.path.append(os.pardir)
sys.path

['',
 '/usr/lib/spark/spark-2.3.2-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip',
 '/usr/lib/spark/spark-2.3.2-bin-hadoop2.7/python',
 '/home/massquantity/Workspace/DL_from_scratch/NOTE',
 '/home/massquantity/.conda/envs/py35/lib/python35.zip',
 '/home/massquantity/.conda/envs/py35/lib/python3.5',
 '/home/massquantity/.conda/envs/py35/lib/python3.5/plat-linux',
 '/home/massquantity/.conda/envs/py35/lib/python3.5/lib-dynload',
 '/home/massquantity/.conda/envs/py35/lib/python3.5/site-packages',
 '/home/massquantity/.conda/envs/py35/lib/python3.5/site-packages/python_recsys-0.2-py3.5.egg',
 '/home/massquantity/.conda/envs/py35/lib/python3.5/site-packages/IPython/extensions',
 '/home/massquantity/.ipython',
 '..']

In [12]:
sys.path.append("/home/massquantity/Workspace/DL_from_scratch/dataset")

In [14]:
from dataset.mnist import load_mnist
from PIL import Image

In [15]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

In [16]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

In [18]:
img = x_train[0].reshape(28, 28) # 将 flatten 后的数据 reshape 为 28*28
label = t_train[0]
img_show(img)

### Cross Entropy

$$
\begin{align*}
& E = -\sum\limits_k t_k \text{log}y_k \\
& E = -\frac{1}{N} \sum\limits_n \sum\limits_k t_{nk} \text{log} y_{nk}  \qquad \text{(batch version)}
\end{align*}
$$

In [10]:
def cross_entropy_error(y, t):  # y, t 均为 one-hot 形式
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(y * np.log(y + 1e-7)) / batch_size

In [11]:
def cross_entropy_error(y, t):  # t 为标签形式 e.g. t = [1,4,8,...], y 为 one-hot
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    if t.size == y.size:   # 若 y 和 t 均为 one-hot， t转为标签
        t = t.argmax(axis=1)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size 
    # y[np.arange(batch_size), t]， batch 中每个样本对应的 t

### Gradient

In [30]:
aa = np.arange(12).reshape(2,3,2)
itt = np.nditer(aa, flags=['multi_index'])
while not itt.finished:
    print(itt.multi_index)
    itt.iternext()

(0, 0, 0)
(0, 0, 1)
(0, 1, 0)
(0, 1, 1)
(0, 2, 0)
(0, 2, 1)
(1, 0, 0)
(1, 0, 1)
(1, 1, 0)
(1, 1, 1)
(1, 2, 0)
(1, 2, 1)


In [38]:
aa = np.arange(12).reshape(2,3,2)
itt = np.nditer(aa, flags=['multi_index'])
itt.remove_multi_index()
print(itt.has_multi_index)
while not itt.finished:
    print(itt.value)
    itt.iternext()

False
0
1
2
3
4
5
6
7
8
9
10
11


In [22]:
def numerical_gradient_2d(f, X):  # 2d matrix
    grad = np.zeros_like(X)
    h = 1e-4
    for idx, x in enumerate(X):
        for idx_2 in range(x.size):
            temp = x[idx_2]
            x[idx_2] = temp + h
            fxh1 = f(x)
            
            x[idx_2] = temp - h
            fxh2 = f(x)
            grad[idx, idx_2] = (fxh1 - fxh2) / (2*h)
            x[idx_2] = temp
    return grad

In [69]:
def numerical_gradient(f, x):
    grad = np.zeros_like(x)
    h = 1e-4
    it = np.nditer(x, flags=['multi_index'], op_flags=["readwrite"])
    while not it.finished:
        idx = it.multi_index
        temp = x[idx]
        x[idx] = temp + h
        fxh1 = f(x)
        
        x[idx] = temp - h
        fxh2 = f(x)
        x[idx] = temp
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        it.iternext()
    return grad

In [70]:
def function_2(x):
    return np.sum(x[0] ** 2 + x[1] ** 2)

In [71]:
aa = np.array([[3.0, 4.0], [3.0, 4.0]])
aa

array([[3., 4.],
       [3., 4.]])

In [72]:
function_2(aa)

50.0

In [73]:
numerical_gradient_2d(function_2, aa)

array([[6., 8.],
       [6., 8.]])

In [74]:
numerical_gradient(function_2, aa)

array([[6., 8.],
       [6., 8.]])

------

### 加法结点的反向传播

![](https://raw.githubusercontent.com/massquantity/DL_from_scratch_NOTE/master/pic/ch05_1.png)

In [3]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x + y
        return out
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

### 乘法结点的反向传播

![](https://raw.githubusercontent.com/massquantity/DL_from_scratch_NOTE/master/pic/ch05_2.png)

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
        
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

### ReLU

In [4]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx =dout
        return dx

### Sigmoid

In [5]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1 - self.out) * self.out
        return dx

### Affine 层

![](https://raw.githubusercontent.com/massquantity/DL_from_scratch_NOTE/master/pic/ch05_3.png)

In [7]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.original_x_shape = None  # 考虑多维数据
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        out = np.dot(self.x, self.W) + b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)
        return dx

### Softmax + Cross Entropy 层

![](https://raw.githubusercontent.com/massquantity/DL_from_scratch_NOTE/master/pic/ch05_4.png)

In [9]:
class SoftmaxWIthLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size:  # t 和 y 均为 one-hot
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), t] -= 1
            dx = dx / batch_size
        
        return dx

### 2 层神经网络的实现

In [10]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        self.params["W2"] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params["b2"] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers["Affine1"] = Affine(self.params['W1'], self.params['b1'])
        self.layers["Relu1"] = Relu()
        self.layers["Affine2"] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = t.argmax(axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

### Gradient Check

In [15]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.allclose(grad_backprop[key], grad_numerical[key])
    print(key + ":" + str(diff))

W2:True
b1:True
b2:True
W1:True


---------

In [ ]:
import os, sys
sys.path.append(os.pardir)
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient

In [7]:
class MultiLayerNet:
    def __init__(self, input_size, hidden_size_list, output_size, 
                 activation='relu', weight_init_std='relu', weight_decay_lambda=0):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size_list = hidden_size_list
        self.hidden_layer_num = len(hidden_size_list)
        self.weight_decay_lambda = weight_decay_lambda
        self.params = {}
        
        self.__init_weight(weight_init_std)
        
        activation_layer = {"relu": Relu, "sigmoid": Sigmoid}
        self.layers = OrderedDict()
        for idx in range(1, self.hidden_layer_num + 1):
            self.layers['Affine' + str(idx)] = Affine(self.params['W' + str(idx)],
                                                      self.params['b' + str(idx)])
            self.layers['Activation_function' + str(idx)] = activation_layer[activation]()
            
        idx = self.hidden_layer_num + 1
        self.layers['Affine' + str(idx)] = Affine(self.params['W' + str(idx)], 
                                                  self.params['b' + str(idx)])
        self.last_layer = SoftmaxWithLoss()
        
    def __init_weight(self, weight_init_std):
        all_size_list = [self.input_size] + self.hidden_size_list + [self.output_size]
        for idx in range(1, len(all_size_list)):
            scale = weight_init_std
            if str(weight_init_std) in ('relu', 'he'):
                scale = np.sqrt(2.0 / all_size_list[idx - 1])
            elif str(weight_init_std) in ('sigmoid', 'xavier'):
                scale = np.sqrt(1.0 / all_size_list[idx - 1])
            
            self.params['W' + str(idx)] = scale * np.random.randn(all_size_list[idx-1], 
                                                                  all_size_list[idx])
            self.params['b' + str(idx)] = np.zeros(all_size_list[idx])
            
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        
        weight_decay = 0
        for idx in range(1, self.hidden_layer_num + 2):
            W = self.params['W' + str(idx)]
            weight_decay += 0.5 * self.weight_decay_lambda * np.sum(W ** 2)
            
        return self.last_layer.forward(y, t) + weight_decay
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = t.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        for idx in range(1, self.hidden_layer_num + 2):
            grads['W' + str(idx)] = numerical_gradient(loss_W, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_W, self.params['b' + str(idx)])
            
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        for idx in range(1, self.hidden_layer_num + 2):
            grads['W' + str(idx)] = self.layers['Affine' + str(idx)].dW + \
                                    self.weight_decay_lambda * self.layers['Affine' + str(idx)].W
            grads['b' + str(idx)] = self.layers['Affine' + str(idx)].db
            
        return grads

----

In [29]:
class MultiLayerNet:
    def __init__(self, input_size, hidden_size_list, output_size, 
                 activation='relu', weight_init_std='relu', weight_decay_lambda=0, 
                 use_dropout=False, dropout_ration=0.5, use_batchnorm=False):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size_list = hidden_size_list
        self.hidden_layer_num = len(hidden_size_list)
        self.use_dropout = use_dropout
        self.weight_decay_lambda = weight_decay_lambda
        slef.use_batchnorm = use_batchnorm
        self.params = {}
        
        self.__init_weight(weight_init_std)
        
        activation_layer = {"relu": Relu, "sigmoid": Sigmoid}
        self.layers = OrderedDict()
        for idx in range(1, self.hidden_layer_num + 1):
            self.layers['Affine' + str(idx)] = Affine(self.params['W' + str(idx)],
                                                      self.params['b' + str(idx)])
            
            if self.use_batchnorm:
                self.params['gamma' + str(idx)] = np.ones(hidden_size_list[idx-1])
                self.params['beta' + str(idx)] = np.zeros(hidden_size_list[idx-1])
                self.layers['Batchnorm' + str(idx)] = BatchNormalization(self.params['gamma' + str(idx)], 
                                                                         self.params['beta' + str(idx)])
            
            self.layers['Activation_function' + str(idx)] = activation_layer[activation]()
            
            if self.use_dropout:
                self.layers['Dropout' + str(idx)] = Dropout(dropout_ration)
            
        idx = self.hidden_layer_num + 1
        self.layers['Affine' + str(idx)] = Affine(self.params['W' + str(idx)], 
                                                  self.params['b' + str(idx)])
        self.last_layer = SoftmaxWithLoss()
        
    def __init_weight(self, weight_init_std):
        all_size_list = [self.input_size] + self.hidden_size_list + [self.output_size]
        for idx in range(1, len(all_size_list)):
            scale = weight_init_std
            if str(weight_init_std) in ('relu', 'he'):
                scale = np.sqrt(2.0 / all_size_list[idx - 1])
            elif str(weight_init_std) in ('sigmoid', 'xavier'):
                scale = np.sqrt(1.0 / all_size_list[idx - 1])
            
            self.params['W' + str(idx)] = scale * np.random.randn(all_size_list[idx-1], 
                                                                  all_size_list[idx])
            self.params['b' + str(idx)] = np.zeros(all_size_list[idx])
            
    def predict(self, x):
        for key, layer in self.layers.items():
            if "Dropout" in key or "BatchNorm" in key:
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        
        weight_decay = 0
        for idx in range(1, self.hidden_layer_num + 2):
            W = self.params['W' + str(idx)]
            weight_decay += 0.5 * self.weight_decay_lambda * np.sum(W ** 2)
            
        return self.last_layer.forward(y, t) + weight_decay
    
    def accuracy(self, x, t):
        y = self.predict(x, train_flg=False)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = t.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, X, T):
        loss_W = lambda W: self.loss(X, T, train_flg=True)

        grads = {}
        for idx in range(1, self.hidden_layer_num+2):
            grads['W' + str(idx)] = numerical_gradient(loss_W, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_W, self.params['b' + str(idx)])
            
            if self.use_batchnorm and idx != self.hidden_layer_num+1:
                grads['gamma' + str(idx)] = numerical_gradient(loss_W, self.params['gamma' + str(idx)])
                grads['beta' + str(idx)] = numerical_gradient(loss_W, self.params['beta' + str(idx)])

        return grads
        
    def gradient(self, x, t):
        self.loss(x, t, train_flg=True)

        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        for idx in range(1, self.hidden_layer_num+2):
            grads['W' + str(idx)] = self.layers['Affine' + str(idx)].dW + self.weight_decay_lambda * self.params['W' + str(idx)]
            grads['b' + str(idx)] = self.layers['Affine' + str(idx)].db

            if self.use_batchnorm and idx != self.hidden_layer_num+1:
                grads['gamma' + str(idx)] = self.layers['BatchNorm' + str(idx)].dgamma
                grads['beta' + str(idx)] = self.layers['BatchNorm' + str(idx)].dbeta

        return grads

In [32]:
class Trainer:
    def __init__(self, network, x_train, t_train, x_test, t_test,
                 epochs=20, mini_batch_size=100,
                 optimizer='SGD', optimizer_param={'lr':0.01}, 
                 evaluate_sample_num_per_epoch=None, verbose=True):
        self.network = network
        self.verbose = verbose
        self.x_train = x_train
        self.t_train = t_train
        self.x_test = x_test
        self.t_test = t_test
        self.epochs = epochs
        self.batch_size = mini_batch_size
        self.evaluate_sample_num_per_epoch = evaluate_sample_num_per_epoch

        optimizer_class_dict = {'sgd':SGD, 'momentum':Momentum, 'nesterov':Nesterov,
                                'adagrad':AdaGrad, 'rmsprpo':RMSprop, 'adam':Adam}
        self.optimizer = optimizer_class_dict[optimizer.lower()](**optimizer_param)
        
        self.train_size = x_train.shape[0]
        self.iter_per_epoch = max(self.train_size / mini_batch_size, 1)
        self.max_iter = int(epochs * self.iter_per_epoch)
        self.current_iter = 0
        self.current_epoch = 0
        
        self.train_loss_list = []
        self.train_acc_list = []
        self.test_acc_list = []

    def train_step(self):
        batch_mask = np.random.choice(self.train_size, self.batch_size)
        x_batch = self.x_train[batch_mask]
        t_batch = self.t_train[batch_mask]
        
        grads = self.network.gradient(x_batch, t_batch)
        self.optimizer.update(self.network.params, grads)
        
        loss = self.network.loss(x_batch, t_batch)
        self.train_loss_list.append(loss)
        if self.verbose: print("train loss:" + str(loss))
        
        if self.current_iter % self.iter_per_epoch == 0:
            self.current_epoch += 1
            
            x_train_sample, t_train_sample = self.x_train, self.t_train
            x_test_sample, t_test_sample = self.x_test, self.t_test
            if not self.evaluate_sample_num_per_epoch is None:
                t = self.evaluate_sample_num_per_epoch
                x_train_sample, t_train_sample = self.x_train[:t], self.t_train[:t]
                x_test_sample, t_test_sample = self.x_test[:t], self.t_test[:t]
                
            train_acc = self.network.accuracy(x_train_sample, t_train_sample)
            test_acc = self.network.accuracy(x_test_sample, t_test_sample)
            self.train_acc_list.append(train_acc)
            self.test_acc_list.append(test_acc)

            if self.verbose: 
                print("=== epoch:" + str(self.current_epoch) + ", train acc:" + str(train_acc) + \
                      ", test acc:" + str(test_acc) + " ===")
        self.current_iter += 1

    def train(self):
        for i in range(self.max_iter):
            self.train_step()

        test_acc = self.network.accuracy(self.x_test, self.t_test)

        if self.verbose:
            print("=============== Final Test Accuracy ===============")
            print("test acc:" + str(test_acc))